In [1]:
# https://www.statistichesulcalcio.com/confronto_squadre_u.php

In [1]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from models import *
from logger import get_logger
from datetime import datetime

import urllib.request
import re
import time
import sys
import pandas as pd

In [2]:
logger = get_logger('StatistichePartite')

In [3]:
url = 'https://www.statistichesulcalcio.com/confronto_squadre_u.php'
evento = 'U2.5'

db.create_tables([Risultati])

In [4]:
now = (datetime.now()).strftime('%Y-%m-%d')

current_ts_run = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [5]:
logger.info("+-----------------------------+")
logger.info("| STARTING StatistichePartite |")
logger.info("+-----------------------------+")

[2021-02-24 09:36:56,969]:[StatistichePartite]:[<module>]:[INFO]: +-----------------------------+
[2021-02-24 09:36:56,974]:[StatistichePartite]:[<module>]:[INFO]: | STARTING StatistichePartite |
[2021-02-24 09:36:56,976]:[StatistichePartite]:[<module>]:[INFO]: +-----------------------------+


In [6]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920,1080")

In [7]:
try:
    logger.info('Avvio Selenium Chrome')
    driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=chrome_options)

except Exception as e:
    
    logger.error(e)
    raise

[2021-02-24 09:37:01,702]:[StatistichePartite]:[<module>]:[INFO]: Avvio Selenium Chrome
/opt/jupyterhub/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
logger.info('Navigo %s' % (url,))
driver.get(url)

[2021-02-24 09:37:09,140]:[StatistichePartite]:[<module>]:[INFO]: Navigo https://www.statistichesulcalcio.com/confronto_squadre_u.php


In [9]:
try:
    logger.info('Estraggo la tabella delle statistiche')

    table_node = driver.find_element_by_id('modificante')
    table = table_node.get_attribute('innerHTML')
    table_soup = BeautifulSoup(table, 'lxml')
except Exception as e:
    
    logger.error(e)
    

[2021-02-24 09:37:30,194]:[StatistichePartite]:[<module>]:[INFO]: Estraggo la tabella delle statistiche


In [10]:
try:

    logger.info('Parsing della tabella delle statistiche')
    stat = (pd.read_html(table)[0])
    stat = stat[stat['POS'].str.match(r'^[0-9]+$')]
    stat = stat[['CAMPIONATO', 'SQUADRA', '%', 'AGGIOR']]

    stat.columns = ['campionato', 'squadra', 'performance', 'data_aggiornamento']

    stat['evento'] = evento
    stat['data_run'] = current_ts_run

except Exception as e:
    
    logger.error(e)
    

[2021-02-24 09:37:33,737]:[StatistichePartite]:[<module>]:[INFO]: Parsing della tabella delle statistiche


In [11]:
logger.info('Inserisco le statistiche in tabella')

for ix, item in stat.to_dict('index').items():
    
    try:
        
        Statistiche.create(**item)
        
    except Exception as e:
        
        logger.error(e)
        
        continue
        

[2021-02-24 09:37:56,356]:[StatistichePartite]:[<module>]:[INFO]: Inserisco le statistiche in tabella
/opt/jupyterhub/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'performance' at row 1")
  result = self._query(query)


In [12]:
def parsing_row(r):
    
    data_match = r[0]
    squadre = [x.strip() for x in (' '+r[2]+' ').split(' - ')]
    risultato = [int(x.strip()) for x in r[3].split(' - ')]
    risultato_pt = [int(x.strip()) for x in r[4].replace('(', '').replace(')', '').split(' - ')]

    d = {
        'data':data_match,
        'squadra_casa': squadre[0],
        'squadra_trasferta': squadre[1],
        'risultato_casa': risultato[0],
        'risultato_trasferta': risultato[1],
        'risultato_pt_casa': risultato_pt[0],
        'risultato_pt_trasferta': risultato_pt[1],
    }
    
    return d
    

In [13]:
t = table_node.find_elements_by_tag_name('tr')

In [14]:
# itero per ogni t

nav_element = []

for tr in t:
    row_element = tr.find_elements_by_tag_name('td')
    
    if row_element!=[]:
        try:
            campionato = row_element[1].text.strip()

            logger.info(campionato)

            dettaglio_url = (row_element[2].find_element_by_tag_name('a')).get_attribute('href')

            logger.info(dettaglio_url)
            
            nav_element.append({'campionato':campionato, 'url':dettaglio_url})
            
        except Exception as e:
            
            logger.error(e)
            
            


[2021-02-24 09:38:26,280]:[StatistichePartite]:[<module>]:[INFO]: ITALY: Serie C - Group C
[2021-02-24 09:38:26,315]:[StatistichePartite]:[<module>]:[INFO]: https://www.statistichesulcalcio.com/dettaglio_partite/Potenza/1587/anno_119.html
[2021-02-24 09:38:26,350]:[StatistichePartite]:[<module>]:[INFO]: INGHILTERRA: Premier League
[2021-02-24 09:38:26,373]:[StatistichePartite]:[<module>]:[INFO]: https://www.statistichesulcalcio.com/dettaglio_partite/Fulham/18/anno_119.html
[2021-02-24 09:38:26,410]:[StatistichePartite]:[<module>]:[INFO]: FRANCIA: Ligue 2
[2021-02-24 09:38:26,439]:[StatistichePartite]:[<module>]:[INFO]: https://www.statistichesulcalcio.com/dettaglio_partite/Le-Havre/375/anno_119.html
[2021-02-24 09:38:26,478]:[StatistichePartite]:[<module>]:[INFO]: KAZAKHSTAN: Premier League
[2021-02-24 09:38:26,504]:[StatistichePartite]:[<module>]:[INFO]: https://www.statistichesulcalcio.com/dettaglio_partite/Taraz/2810/anno_119.html
[2021-02-24 09:38:26,540]:[StatistichePartite]:[<mod

In [15]:
current_ts_run = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for el in nav_element:
    logger.info('%s - %s' % (el['campionato'], el['url']))
    driver.get(el['url'])
    logger.info('Aspetto 1 sec')
    time.sleep(5)
            
    dettaglio_table = driver.find_elements_by_class_name('table-responsive')
    
    cnt = 0
    
    for tbl in dettaglio_table:
        cnt += 1
        logger.info('Tabella risultati n %d' % cnt)
        
        dettaglio_df = pd.read_html(tbl.get_attribute('innerHTML'))
        
        if dettaglio_df!=[]:
            
            dettaglio_df = dettaglio_df[0]
            logger.info('\n'+dettaglio_df.__repr__())
            for ix, row in dettaglio_df.iterrows():

                single_dict = parsing_row(row)
                single_dict['campionato'] = el['campionato']
                single_dict['data_run'] = current_ts_run

                Risultati.create(**single_dict)    
    logger.info('Completato %s' % el['url'])       

[2021-02-24 09:38:38,346]:[StatistichePartite]:[<module>]:[INFO]: ITALY: Serie C - Group C - https://www.statistichesulcalcio.com/dettaglio_partite/Potenza/1587/anno_119.html
[2021-02-24 09:38:41,144]:[StatistichePartite]:[<module>]:[INFO]: Aspetto 1 sec
[2021-02-24 09:38:42,229]:[StatistichePartite]:[<module>]:[INFO]: Tabella risultati n 1
[2021-02-24 09:38:42,354]:[StatistichePartite]:[<module>]:[INFO]: 
            0                   1                             2      3  \
0  21-02-2021  Lega Pro - Group C  Potenza - Virtus Francavilla  0 - 0   
1  17-02-2021  Lega Pro - Group C              Teramo - Potenza  1 - 3   
2  07-02-2021  Lega Pro - Group C              Potenza - Foggia  0 - 1   
3  03-02-2021  Lega Pro - Group C           Casertana - Potenza  1 - 0   
4  30-01-2021  Lega Pro - Group C             Potenza - Palermo  0 - 0   
5  23-01-2021  Lega Pro - Group C           Catanzaro - Potenza  1 - 0   
6  10-01-2021  Lega Pro - Group C            Potenza - Paganese  1 - 0  

WebDriverException: Message: unknown error: net::ERR_CONNECTION_CLOSED
  (Session info: headless chrome=87.0.4280.66)


In [16]:
driver.close()